In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings(action = "ignore")

In [2]:
from datetime import timedelta, datetime
import glob
from itertools import chain
import json
import os
import re

import numpy as np
import pandas as pd

from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from konlpy.tag import Twitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from pandas.plotting import register_matplotlib_converters
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
# font_path = '/usr/share/fonts/NanumGothic.ttf'
# font_name = fm.FontProperties(fname=font_path, size=10).get_name()
# plt.rc('font', family=font_name, size=12)
plt.rcParams["figure.figsize"] = (20, 10)
register_matplotlib_converters()

mpl.font_manager._rebuild()
# mpl.pyplot.rc('font', family='NanumGothic')

- 학습 데이터: 총 115,071개 플레이리스트 정보가 담겨져 있으며 포함하는 값은 아래와 같습니다.
    - tags : 플레이리스트에 매핑된 태그
    - id : 플레이리스트 아이디
    - plylst_title : 플레이리스트 명
    - songs : 플레이리스트 내 수록된 곡 아이디
    - like_cnt : 플레이리스트 좋아요 횟수
    - updt_date : 플레이리스트 업데이트 일시

- 왜 곡명, 장르, 플레이리스트 태그가 중요하다고 생각했는가?
    - train 데이터셋의 주요한 속성: 플레이리스트 아이디, 플레이리스트에 매핑된 태그, 플레이리스트에 수록된 곡 정보
    - 이것을 통해 플레이리스트에 들어갈 곡명(리스트), 플레이리스트의 태그(리스트)를 예측
    - 곡에는? 총 707,989곡의 메타 정보 
        - id : 곡 아이디
        - song_name : 곡 명
        - song_gn_gnr_basket : 대분류 장르코드
        - song_gn_dtl_gnr_basket : 상세 장르 코드
        - album_id : 앨범 아이디
        - album_name : 앨범 명
        - artist_name_basket : 아티스트 명 (복수일 경우 띄어쓰기로 구분)
        - artist_id_basket : 아티스트 아이디 (복수일 경우 띄어쓰기로 구분)
        - issue_date : 곡 발매 일자 (yyyymmdd)
    - 살펴보면, 곡에서 주요한 속성은 곡명, 장르, 앨범, 아티스트로 나눌 수 있다
    
    - 곡추천: 해당 플레이리스트에 포함된 곡과 유사한 장르, 앨범, 아티스트를 가진 곡 추천
    - 태그추천: 곡 추천의 결과를 바탕으로 (이미 유사한 곡들을 추려냈으므로), 그 곡들에 맵핑되는 태그들을 추천

- 필요한 것: train 세트에 포함된 플레이리스트 수록된 song id 리스트 - 각 song의 정보를 얻기 위해 song_meta 테이블과 조인
    - 그러면 플레이리스트 곡별 장르, 앨범, 아티스트 정보를 구할 수 있음

- 우리의 목표: 가지고 있는 데이터를 통해 태그명, 곡명을 예측

- 가지고 있는 데이터셋에서 연관성을 파악
    - 가장 주요한 연관성: 플레이리스트 태그와 곡 장르의 상관관계, 플레이리스트 태그와 곡명의 상관관계
    - 태그로 곡 장르 파악 가능한 경우 있음
        
    
- 태그 예측: 분류
    - 예측: 주요 태그 추출해서 다중분류 형식으로
    - 신경망 구축? (동현)
    - 주요태그를 추출하는 기준은?
    
- 곡 예측: 회귀? 분류?
    - 곡명을 어떻게 예측하나? 다중분류 (곡의 주요 값)
    - 곡을 키워드로 군집화(형태소 분석)
    - 태그로 장르 파악 -> 같은 장르를 가진 곡

- 플레이리스트 태그가 곡 장르인 경우가 있음
- 태그명으로 장르 파악도 가능할듯
- like_cnt, updt_year에 따라 train/test/val 이 분포되어 있다
    - 중요한 정보일 수 있으나, 이걸로 train과 test를 구분하려고 하면 의미가 없음(똑같으므로)

In [3]:
df_song = pd.read_json("./song_meta.json")

In [4]:
df_song_info = df_song[['id' , 'song_name' , 'artist_id_basket', 'artist_name_basket' , 'album_id' , 'album_name' , 'song_gn_gnr_basket' , 'song_gn_dtl_gnr_basket']]

In [ ]:
df_song_info

In [5]:
df_song_info['genre_literal'] = df_song_info.song_gn_dtl_gnr_basket.apply(lambda x : (' ').join(x))

In [6]:
df_song_info['artist_literal'] = df_song_info.artist_id_basket.apply(lambda x : str(x[0]))

In [7]:
df_song_info['album_literal'] = df_song_info['album_id'].apply(lambda x : str(x))

In [8]:
# 장르, 아티스트, 앨범 정보를 모두 한 열에 포함(유사도 측정)

df_song_info['all'] = df_song_info['genre_literal'] + " " + df_song_info['artist_literal'] + " " + df_song_info['album_literal']
        

In [9]:
df_song_info['all']

0                  GN0901 2727 2255639
1           GN1601 GN1606 29966 376431
2                  GN0901 3361 4698747
3         GN1102 GN1101 838543 2644882
4         GN1802 GN1801 560160 2008470
                      ...             
707984             GN2001 166499 65254
707985              GN0901 11837 44141
707986       GN0105 GN0101 437 2662866
707987    GN1807 GN1801 729868 2221722
707988         GN0601 GN0604 895 34663
Name: all, Length: 707989, dtype: object

In [10]:
# 5만개로 코사인유사도 측정

df_song_info_1 = df_song_info[:50000]

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))

song_mat_1 = count_vect.fit_transform(df_song_info_1['all'])

song_sim_1 = cosine_similarity(song_mat_1, song_mat_1)

song_sim_sorted_ind_1 = song_sim_1.argsort()[: , ::-1]

In [ ]:
song_mat

In [ ]:
song_sim_1

In [ ]:
a = song_sim_1[1].sort()

In [52]:
def find_sim_song(song_id, top_n = 10) :
    
    # 인자로 입력된 df_song_info 에서 song_name 추출
    song_id = df_song_info_1[df_song_info_1['id'] == song_id]
    
    # index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    id_index = song_id.index.values
    similar_indexes = song_sim_sorted_ind_1[id_index, :(top_n)]
    
    
    # 추출된 top_n index 출력
    # 데이터프레임에서 index로 사용하기 위해 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df_song_info_1.iloc[similar_indexes]

In [53]:
find_sim_song(1, 10)

[[    1  1078  6688 29430 16884 48111 38892 18394 47288 12319]]


,id,song_name,artist_id_basket,artist_name_basket,album_id,album_name,song_gn_gnr_basket,song_gn_dtl_gnr_basket,genre_literal,artist_literal,album_literal,all
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[29966],[Murray Perahia],376431,"Bach : Partitas Nos. 2, 3 & 4",[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,376431,GN1601 GN1606 29966 376431
1078,1078,Schumann : Sonata For Piano No.1 In F- Sharp M...,[29966],[Murray Perahia],48317,"Schumann : Piano Sonata, Op.11 & Kreisleriana,...",[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,48317,GN1601 GN1606 29966 48317
6688,6688,"J.S. Bach: French Suite No.1 in D Minor, BWV 8...",[29966],[Murray Perahia],10004046,Johann Sebastian Bach : The French Suites,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,10004046,GN1601 GN1606 29966 10004046
29430,29430,"Chopin : Barcarolle In F-Sharp Major, Op.60",[29966],[Murray Perahia],47306,Chopin : Impromptus,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,47306,GN1601 GN1606 29966 47306
16884,16884,"Bach : English Suite No.6 In D Minor, BWV811 -...",[29966],[Murray Perahia],49708,"Bach : English Suites Nos.1, 3 & 6",[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,49708,GN1601 GN1606 29966 49708
48111,48111,Brahms : Handel Variations Op.24 (브람스 : 헨델 변주곡...,[29966],[Murray Perahia],1072776,Brahms : Handel Variations,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,1072776,GN1601 GN1606 29966 1072776
38892,38892,"J.S. Bach: French Suite No.5 In G Major, BWV 8...",[29966],[Murray Perahia],10004046,Johann Sebastian Bach : The French Suites,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,10004046,GN1601 GN1606 29966 10004046
18394,18394,"Bach : Goldberg Variations, BWV988 - Aria",[29966],[Murray Perahia],49719,Bach : Goldberg Variations,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,49719,GN1601 GN1606 29966 49719
47288,47288,Schubert : Piano Quintet In A Major Op.114 D.6...,[828609],[클래식코드],2306626,로맨틱 분위기에 꼭 필요한 클래식 2,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,828609,2306626,GN1601 GN1606 828609 2306626
12319,12319,Trifonov: Rachmaniana - 4. Dolce romantico,[742100],[Daniil Trifonov],2337142,Rachmaninov Variations,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,742100,2337142,GN1601 GN1606 742100 2337142


In [ ]:
pd.DataFrame(df_song_info.iloc[1])

In [ ]:
song_id = df_song_info_1[df_song_info_1['id'] == 2]

In [ ]:
song_id

In [ ]:
id_index = song_id.index.values

In [ ]:
id_index

In [ ]:
similar_indexes = song_sim_sorted_ind_1[id_index, :10]

In [ ]:
similar_indexes.reshape(-1)

In [ ]:
def find_sim_song(df, sorted_ind, song_id, top_n = 10) :
    
    # 인자로 입력된 df_song_info 에서 song_name 추출
    song_id = df[df['id'] == song_id]
    
    # index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    id_index = song_id.index.values
    similar_indexes = sorted_ind[id_index, :(top_n)]
    
    
    # 추출된 top_n index 출력
    # 데이터프레임에서 index로 사용하기 위해 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [ ]:
df_song_info_1 = df_song_info[:50000]
df_song_info_2 = df_song_info[50000:100000]
df_song_info_3 = df_song_info[100000:150000]
df_song_info_4 = df_song_info[150000:200000]
df_song_info_5 = df_song_info[200000:250000]
df_song_info_6 = df_song_info[250000:300000]
df_song_info_7 = df_song_info[300000:350000]
df_song_info_8 = df_song_info[350000:400000]
df_song_info_9 = df_song_info[400000:450000]
df_song_info_10 = df_song_info[450000:500000]
df_song_info_11 = df_song_info[500000:550000]
df_song_info_12 = df_song_info[550000:600000]
df_song_info_13 = df_song_info[600000:650000]
df_song_info_14 = df_song_info[650000:700000]
df_song_info_15 = df_song_info[700000:]

In [ ]:
# append 로 리스트 합치기


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 반환
count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))


def find_sim_song(df, sorted_ind, song_id, top_n = 10) :
    
    # 인자로 입력된 df_song_info 에서 song_name 추출
    song_id = df[df['id'] == song_id]
    
    # index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    id_index = song_id.index.values
    similar_indexes = sorted_ind[id_index, :(top_n)]
    
    
    # 추출된 top_n index 출력
    # 데이터프레임에서 index로 사용하기 위해 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]




df_top10 = []
for i in range(len(df_song_info)//50000+1) : 
    
    df_song_info_i = df_song_info[i*50000:(i+1)*50000]
    genre_mat_i = count_vect.fit_transform(df_song_info_i['all'])
    genre_sim_i = cosine_similarity(genre_mat_i, genre_mat_i)
    genre_sim_sorted_ind_i = genre_sim_i.argsort()[: , ::-1]
    top10 = genre_sim_sorted_ind_i[:11]
    df_top10.append(top10)
    

print(df_top10)
    
#     print(f"df_song_info_{i}")
#     print("="*50)
#     print(df_song_info_i.head())
#     print("="*50)




In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# df_song_info_1 = df_song_info[:50000]
# df_song_info_2 = df_song_info[50000:100000]
# df_song_info_3 = df_song_info[100000:150000]
# df_song_info_4 = df_song_info[150000:200000]
# df_song_info_5 = df_song_info[200000:250000]
# df_song_info_6 = df_song_info[250000:300000]
# df_song_info_7 = df_song_info[300000:350000]
# df_song_info_8 = df_song_info[350000:400000]
# df_song_info_9 = df_song_info[400000:450000]
# df_song_info_10 = df_song_info[450000:500000]
# df_song_info_11 = df_song_info[500000:550000]
# df_song_info_12 = df_song_info[550000:600000]
# df_song_info_13 = df_song_info[600000:650000]
# df_song_info_14 = df_song_info[650000:700000]
# df_song_info_15 = df_song_info[700000:]


# count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))
# song_mat_1 = count_vect.fit_transform(df_song_info_1['all'])
# song_mat_2 = count_vect.fit_transform(df_song_info_2['all'])
# song_mat_3 = count_vect.fit_transform(df_song_info_3['all'])
# song_mat_4 = count_vect.fit_transform(df_song_info_4['all'])
# song_mat_5 = count_vect.fit_transform(df_song_info_5['all'])
# song_mat_6 = count_vect.fit_transform(df_song_info_6['all'])
# song_mat_7 = count_vect.fit_transform(df_song_info_7['all'])
# song_mat_8 = count_vect.fit_transform(df_song_info_8['all'])
# song_mat_9 = count_vect.fit_transform(df_song_info_9['all'])
# song_mat_10 = count_vect.fit_transform(df_song_info_10['all'])
# song_mat_11 = count_vect.fit_transform(df_song_info_11['all'])
# song_mat_12 = count_vect.fit_transform(df_song_info_12['all'])
# song_mat_13 = count_vect.fit_transform(df_song_info_13['all'])
# song_mat_14 = count_vect.fit_transform(df_song_info_14['all'])
# song_mat_15 = count_vect.fit_transform(df_song_info_15['all'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity



count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))

In [ ]:
df_top10 = []

song_mat_1 = count_vect.fit_transform(df_song_info_1['all'])
song_sim_1 = cosine_similarity(song_mat_1, song_mat_1)
song_sim_sorted_ind_1 = song_sim_1.argsort()[: , ::-1][:, :10]
df_top10.append(song_sim_sorted_ind_1)


print(df_top10)

In [ ]:
df_top10

In [ ]:
song_mat_2 = count_vect.fit_transform(df_song_info_2['all'])
song_sim_2 = cosine_similarity(song_mat_2, song_mat_2)
song_sim_sorted_ind_2 = song_sim_2.argsort()[: , ::-1][:, :10]
df_top10.append(song_sim_sorted_ind_2)


print(df_top10)

In [ ]:
song_sim_sorted_ind_1[:, :10]

In [ ]:
genre_sim_sorted_ind_1 = genre_sim_1.argsort()[: , ::-1]

In [ ]:
# 유사도가 높은 순으로 정렬 (0~49999)
genre_sim_sorted_ind_1

In [48]:
def find_sim_song(df, sorted_ind, song_id, top_n = 10) :
    
    # 인자로 입력된 df_song_info 에서 song_name 추출
    song_id = df[df['id'] == song_id]
    
    # index 객체를 ndarray로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    id_index = song_id.index.values
    similar_indexes = sorted_ind[id_index, :(top_n)]
    
    
    # 추출된 top_n index 출력
    # 데이터프레임에서 index로 사용하기 위해 1차원 array로 변경
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [ ]:
find_sim_song(df_song_info_1, genre_sim_sorted_ind_1, 1, 10)

In [ ]:
df_plylst = pd.read_json("./test.json")

In [ ]:
df_plylst.head()

In [ ]:
df_plylst.info()

In [ ]:





from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity



count_vect = CountVectorizer(min_df = 0, ngram_range = (1,2))



song_mat_1 = count_vect.fit_transform(df_song_info_1['all'])
song_sim_1 = cosine_similarity(song_mat_1, song_mat_1)
song_sim_sorted_ind_1 = song_sim_1.argsort()[: , ::-1][:, :10]
df_top10.append(song_sim_sorted_ind_1)





In [71]:
pd.DataFrame(df_song_info.iloc[1])

,1
id,1
song_name,"Bach : Partita No. 4 In D Major, BWV 828 - II...."
artist_id_basket,[29966]
artist_name_basket,[Murray Perahia]
album_id,376431
album_name,"Bach : Partitas Nos. 2, 3 & 4"
song_gn_gnr_basket,[GN1600]
song_gn_dtl_gnr_basket,"[GN1601, GN1606]"
genre_literal,GN1601 GN1606
artist_literal,29966


In [72]:
find_sim_song(1, 10)

[[    1  1078  6688 29430 16884 48111 38892 18394 47288 12319]]


,id,song_name,artist_id_basket,artist_name_basket,album_id,album_name,song_gn_gnr_basket,song_gn_dtl_gnr_basket,genre_literal,artist_literal,album_literal,all
1,1,"Bach : Partita No. 4 In D Major, BWV 828 - II....",[29966],[Murray Perahia],376431,"Bach : Partitas Nos. 2, 3 & 4",[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,376431,GN1601 GN1606 29966 376431
1078,1078,Schumann : Sonata For Piano No.1 In F- Sharp M...,[29966],[Murray Perahia],48317,"Schumann : Piano Sonata, Op.11 & Kreisleriana,...",[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,48317,GN1601 GN1606 29966 48317
6688,6688,"J.S. Bach: French Suite No.1 in D Minor, BWV 8...",[29966],[Murray Perahia],10004046,Johann Sebastian Bach : The French Suites,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,10004046,GN1601 GN1606 29966 10004046
29430,29430,"Chopin : Barcarolle In F-Sharp Major, Op.60",[29966],[Murray Perahia],47306,Chopin : Impromptus,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,47306,GN1601 GN1606 29966 47306
16884,16884,"Bach : English Suite No.6 In D Minor, BWV811 -...",[29966],[Murray Perahia],49708,"Bach : English Suites Nos.1, 3 & 6",[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,49708,GN1601 GN1606 29966 49708
48111,48111,Brahms : Handel Variations Op.24 (브람스 : 헨델 변주곡...,[29966],[Murray Perahia],1072776,Brahms : Handel Variations,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,1072776,GN1601 GN1606 29966 1072776
38892,38892,"J.S. Bach: French Suite No.5 In G Major, BWV 8...",[29966],[Murray Perahia],10004046,Johann Sebastian Bach : The French Suites,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,10004046,GN1601 GN1606 29966 10004046
18394,18394,"Bach : Goldberg Variations, BWV988 - Aria",[29966],[Murray Perahia],49719,Bach : Goldberg Variations,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,29966,49719,GN1601 GN1606 29966 49719
47288,47288,Schubert : Piano Quintet In A Major Op.114 D.6...,[828609],[클래식코드],2306626,로맨틱 분위기에 꼭 필요한 클래식 2,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,828609,2306626,GN1601 GN1606 828609 2306626
12319,12319,Trifonov: Rachmaniana - 4. Dolce romantico,[742100],[Daniil Trifonov],2337142,Rachmaninov Variations,[GN1600],"[GN1601, GN1606]",GN1601 GN1606,742100,2337142,GN1601 GN1606 742100 2337142
